# Assignment 2

In this assignment the goal is to be able to price a game based on an image. This will be done using a convolutional neural network.
For this assignment three methods are developed with the goal of pricing the games:
1. Building a CNN that does not take into consideration that some screenshots belong to the same game.
1. Building a CNN that takes into consideration that some screenshots belong to the same game by stitching the screenshots of a game together to create one large image. This large image is then fed into the CNN. 
1. Building a CNN that takes into consideration that some screenshots belong to the same game by using multiple screenshots of a game as input of a multi-branch CNN with a concatenated layer.

First, some general pre-processing is discussed. This ensures that the same splits will be used for each method. Then, for each method the pre-processing steps are discussed. Then each model with the necessary evaluation criteria is built. Then, based on the the evaluation metric, the best of the three beforementioned methods is selected. Lastly, some interpretability techniques to get insights on the model is given. 

To start, the necessary packages and data are loaded in. The data with for each game the related price and images are included. Additionally, a path is created to where the images are stored on the local device.

In [ ]:
#Load the following packages:
import os
import json
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import tensorflow as tf


# Set the location of where the images are stored. Adjust to the correct location when running the code!
image_location = '/Users/sarahguilliams/Desktop/Advanced Analytics in a Big Data World/Assignment2/images'

#  Load the json datafile
file_path = "dataset.json"

# Open the JSON file and load its contents
with open(file_path, 'r') as file:
    data = json.load(file)

data1 = pd.DataFrame(data) #Make sure that the data has the DataFrame format
data1.head() #print the first 5 rows with data

## Pre-processing

### Removing outliers

In [ ]:
# Select a portion of the games. If we want to train on all of the data, set it equal to 1.
num_selected_games = int(len(data1))
np.random.seed(0) #This is so that we get the same games when training the model.
selected_games = data1.sample(n=num_selected_games)
#print(selected_games) 

# Remove the games that have a price larger than 100 USD from train_games
selected_games_no_outliers = selected_games[selected_games['price'] <= 10000]



### Train-test split

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
# Split data into training and validation sets based on games
train1_games, test_games = train_test_split(selected_games_no_outliers, test_size=0.2, random_state=42) 
train_games, val_games = train_test_split(train1_games, test_size=0.25, random_state=42) # 0.25 * 0.8 = 0.2